<a href="https://colab.research.google.com/github/Meii0606/Breaking-reCAPTCHAv2/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-414/714: Deep Learning Systems - Final Project


## **Sparse Matrix Multiplication on Graph Neural Network**

**By Andrew Zhang, Jinkai Qiu & Yimei Wu**


---

In this project, we are going to implement **sparse matrix** class supported in Needle, **forward and backward pass of sparse matrix multiplication**, and its **application on Graph Neural Network(GNN)**.

In this notebook, we are going to show how to **define sparse matrix**, perform **sparse matrix multiplication** and compare it with normal dense matrix multiplication. In addition, we will also show how it can be used in **GNN training.**



## 1. Clone Required Repo and Install Required Packages

In [69]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/10714
!mkdir -p final_proj
%cd /content/drive/MyDrive/10714/final_proj
!git clone https://github.com/AndrewZhang76/gnn_with_spmm.git
%cd gnn_with_spmm/
!git pull
!pip3 install pybind11

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/10714
/content/drive/MyDrive/10714/final_proj
Cloning into 'gnn_with_spmm'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 248 (delta 102), reused 193 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (248/248), 707.46 KiB | 6.37 MiB/s, done.
Resolving deltas: 100% (102/102), done.
/content/drive/MyDrive/10714/final_proj/gnn_with_spmm
Already up to date.


## 2. Build

In [1]:
%cd /content/drive/MyDrive/10714/final_proj/gnn_with_spmm/
!make

/content/drive/MyDrive/10714/final_proj/gnn_with_spmm
-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.13.6")
CMake Warning (dev) at CMakeLists.txt:57 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- CUDA_FOUND: TRUE
-- Found cuda, building cuda backend
Wed Dec 11 17:55:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-U

In [2]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


In [3]:
import os
import numpy as np

class CoraDataset:
    """
    A simple class for the Cora dataset.
    Processes the data from .content and .cites files.
    Returns:
        - X: Feature matrix (np.array of shape [num_nodes, num_features])
        - y: Labels (np.array of shape [num_nodes])
        - adjacency_matrix: Dense adjacency matrix (np.array of shape [num_nodes, num_nodes])
    """

    def __init__(self, content_file: str, cites_file: str):
        """
        Initializes the dataset with provided file paths.

        Args:
            content_file (str): Path to the .content file.
            cites_file (str): Path to the .cites file.
        """
        self.content_file = content_file
        self.cites_file = cites_file

        # These will store the processed data
        self.X = None
        self.y = None
        self.adjacency_matrix = None
        self.label_mapping = None

        # Load and process the data
        self._load_data()

    def _load_data(self):
        """Loads and processes node features, labels, and edges."""
        nodes = []
        labels = []
        features = []

        # Read the .content file
        with open(self.content_file, 'r') as f:
            for line in f:
                elements = line.strip().split('\t')
                nodes.append(elements[0])
                features.append([float(x) for x in elements[1:-1]])
                labels.append(elements[-1])

        # Encode labels into integers
        self.label_mapping = {label: idx for idx, label in enumerate(sorted(set(labels)))}
        encoded_labels = [self.label_mapping[label] for label in labels]

        # Convert features and labels to NumPy arrays
        self.X = np.array(features, dtype=np.float32)
        self.y = np.array(encoded_labels, dtype=np.int64)

        # Build the adjacency matrix
        num_nodes = len(nodes)
        node_index = {node: idx for idx, node in enumerate(nodes)}
        self.adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=np.float32)

        with open(self.cites_file, 'r') as f:
            for line in f:
                src, dst = line.strip().split('\t')
                if src in node_index and dst in node_index:
                    i, j = node_index[src], node_index[dst]
                    self.adjacency_matrix[i, j] = 1
                    self.adjacency_matrix[j, i] = 1  # Assume undirected graph

    def get_data(self):
        """Returns the full dataset: features, labels, and adjacency matrix."""
        return self.X, self.y, self.adjacency_matrix

    def get_label_mapping(self):
        """Returns the mapping from original labels to encoded integers."""
        return self.label_mapping

class DataLoader:
    """
    A simple data loader for batching and shuffling CoraDataset data.

    Args:
        X (np.array): Feature matrix.
        y (np.array): Labels.
        adjacency_matrix (np.array): Dense adjacency matrix.
        batch_size (int): Number of samples per batch.
        shuffle (bool): Whether to shuffle the data at the beginning of each iteration.
    """

    def __init__(self, X, y, adjacency_matrix, batch_size=1, shuffle=False):
        self.X = X
        self.y = y
        self.adjacency_matrix = adjacency_matrix
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_samples = X.shape[0]
        self.order = np.arange(self.num_samples)

    def __iter__(self):
        """Initializes the iterator."""
        if self.shuffle:
            np.random.shuffle(self.order)
        self.index = 0
        return self

    def __next__(self):
        """Fetches the next batch of data."""
        if self.index >= self.num_samples:
            raise StopIteration

        # Get indices for the current batch
        start = self.index
        end = min(self.index + self.batch_size, self.num_samples)
        batch_indices = self.order[start:end]

        # Fetch the batch
        X_batch = self.X[batch_indices]
        y_batch = self.y[batch_indices]
        adjacency_batch = self.adjacency_matrix[batch_indices][:, batch_indices]

        self.index += self.batch_size
        return X_batch, y_batch, adjacency_batch

import sys
sys.path.append('./python')
sys.path.append('./apps')
import needle as ndl

def softmax_loss(Z, y_one_hot):
    """Return softmax loss.  Note that for the purposes of this assignment,
    you don't need to worry about "nicely" scaling the numerical properties
    of the log-sum-exp computation, but can just compute this directly.

    Args:
        Z (ndl.Tensor[np.float32]): 2D Tensor of shape
            (batch_size, num_classes), containing the logit predictions for
            each class.
        y (ndl.Tensor[np.int8]): 2D Tensor of shape (batch_size, num_classes)
            containing a 1 at the index of the true label of each example and
            zeros elsewhere.

    Returns:
        Average softmax loss over the sample. (ndl.Tensor[np.float32])
    """
    ### BEGIN YOUR SOLUTION
    m = Z.shape[0]
    exps = ndl.ops.exp(Z)
    log = ndl.ops.log(ndl.ops.summation(exps, axes=(1, )))
    Z1 = ndl.ops.summation(log)
    Z2 = ndl.ops.summation(Z * y_one_hot)
    return (Z1 - Z2) / m
    ### END YOUR SOLUTION

def nn_epoch(X, y, W1, W2, lr=0.1, batch=100):
    """Run a single epoch of SGD for a two-layer neural network defined by the
    weights W1 and W2 (with no bias terms):
        logits = ReLU(X * W1) * W1
    The function should use the step size lr, and the specified batch size (and
    again, without randomizing the order of X).

    Args:
        X (np.ndarray[np.float32]): 2D input array of size
            (num_examples x input_dim).
        y (np.ndarray[np.uint8]): 1D class label array of size (num_examples,)
        W1 (ndl.Tensor[np.float32]): 2D array of first layer weights, of shape
            (input_dim, hidden_dim)
        W2 (ndl.Tensor[np.float32]): 2D array of second layer weights, of shape
            (hidden_dim, num_classes)
        lr (float): step size (learning rate) for SGD
        batch (int): size of SGD mini-batch

    Returns:
        Tuple: (W1, W2)
            W1: ndl.Tensor[np.float32]
            W2: ndl.Tensor[np.float32]
    """

    ### BEGIN YOUR SOLUTION
    m = X.shape[0]
    for i in range(0, m, batch):
        X_batch = X[i : i+batch]
        y_batch = y[i : i+batch]
        X_batch = ndl.Tensor(X_batch)
        Z1 = ndl.ops.relu(X_batch @ W1)
        Z = Z1 @ W2
        y_one_hot = np.zeros(Z.shape, dtype="float32")
        y_one_hot[np.arange(Z.shape[0]),y_batch] = 1
        loss = softmax_loss(Z, ndl.Tensor(y_one_hot))
        loss.backward()
        w1_grad = W1.grad
        w2_grad = W2.grad
        W1 = np.array(W1) - lr * w1_grad
        W2 = np.array(W2) - lr * w2_grad
        W1 = ndl.Tensor(W1)
        W2 = ndl.Tensor(W2)
    return W1, W2
    ### END YOUR SOLUTION

# Training function
def train_gcn(model, dataset, num_epochs, learning_rate, batch_size):
    """
    Train the GCN model on the Cora dataset.

    Args:
        model: The GCN model instance.
        dataset: CoraDataset instance providing data.
        num_epochs: Number of training epochs.
        learning_rate: Learning rate for gradient descent.
        batch_size: Number of samples per batch.

    Returns:
        model: Trained GCN model.
    """
    X, y, adjacency_matrix = dataset.get_data()
    num_samples = X.shape[0]
    num_classes = len(np.unique(y))

    # Convert data to Needle Tensors
    X_tensor = ndl.Tensor(X)
    adjacency_tensor = ndl.Tensor(adjacency_matrix)
    y_one_hot = np.eye(num_classes)[y]  # One-hot encode labels

    for epoch in range(num_epochs):
        epoch_loss = 0
        num_batches = int(np.ceil(num_samples / batch_size))

        for batch_idx in range(0, num_samples, batch_size):
            # Fetch the batch
            batch_end = min(batch_idx + batch_size, num_samples)
            X_batch = X_tensor[batch_idx:batch_end]
            y_batch = ndl.Tensor(y_one_hot[batch_idx:batch_end])

            # Forward pass
            logits = model(X_batch, adjacency_tensor)

            # Compute loss
            loss = softmax_loss(logits, y_batch)

            # Backward pass
            loss.backward()

            # Update weights
            for param in model.parameters():
                param.data -= learning_rate * param.grad.data
                param.grad = None  # Reset gradients

            epoch_loss += loss.data

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / num_batches}")

    return model


# Evaluation function
def evaluate_gcn(model, dataset):
    """
    Evaluate the GCN model on the dataset.

    Args:
        model: The trained GCN model.
        dataset: CoraDataset instance providing data.

    Returns:
        accuracy: Accuracy of the model on the dataset.
    """
    X, y, adjacency_matrix = dataset.get_data()
    num_samples = X.shape[0]

    # Convert data to Needle Tensors
    X_tensor = ndl.Tensor(X)
    adjacency_tensor = ndl.Tensor(adjacency_matrix)

    # Forward pass
    logits = model(X_tensor, adjacency_tensor)
    predictions = np.argmax(logits.data, axis=1)

    # Calculate accuracy
    accuracy = np.mean(predictions == y)
    return accuracy

Using needle backend


ModuleNotFoundError: No module named 'needle.data.datasets.mnist_dataset'

In [81]:
!make

-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.13.6")
CMake Warning (dev) at CMakeLists.txt:57 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- CUDA_FOUND: TRUE
-- Found cuda, building cuda backend
Wed Dec 11 17:51:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                   

In [85]:
import sys
sys.modules.clear()

In [84]:
import needle.nn as nn
# Example usage
content_path = './data/cora/cora.content'
cites_path = './data/cora/cora.cites'
cora_dataset = CoraDataset(content_path, cites_path)

# Initialize GCN model
in_features = cora_dataset.X.shape[1]
hidden_features = 16
out_features = len(np.unique(cora_dataset.y))
print(in_features)
print(out_features)
Conv = nn.Conv
gcn_model = nn.GCN(in_features, hidden_features, out_features)
# Train GCN model
trained_gcn = train_gcn(gcn_model, cora_dataset, num_epochs=100, learning_rate=0.01, batch_size=64)
# Evaluate GCN model
accuracy = evaluate_gcn(trained_gcn, cora_dataset)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

1433
7


AttributeError: module 'needle.nn' has no attribute 'GCN'

## 3. Sparse Matrix Definiation.
In this project, we defined a new way to represent a sparse matrix(a type of matrix that contains a significant number of zero elements compared to the total number of elements in the matrix.) - **COO (Coordinate) format**. \
The COO (Coordinate) format is a representation of a sparse matrix that stores only the nonzero elements along with their row and column indices. It is efficient in terms of memory usage for sparse matrices because it avoids storing zero values.\
### Key Components:


1.   Values (data): A list or array of the nonzero elements in the matrix.
2.   Row indices (row): A list or array specifying the row index for each nonzero element.
3. Column indices (col): A list or array specifying the column index for each nonzero element.

###Properties:
1. Flexible: Allows easy manipulation, such as matrix construction from nonzero entries.
2. Duplicates: COO format allows duplicate entries. To obtain the actual matrix, these duplicates need to be summed.
3. Conversion: Often converted to other formats like CSR (Compressed Sparse Row) or CSC (Compressed Sparse Column) for efficient matrix operations.






#### First, we are going to randomly generate a sparse matrix in normal NDArray format. It is a 10 * 10 matrix with 10 non-zero elements.

In [ ]:
%cd /content/drive/MyDrive/10714/final_proj/gnn_with_spmm/python/needle
import numpy as np
from backend_ndarray.ndarray import *

np.random.seed(0)
device = cuda()
# Dimensions of the matrix
rows, cols = 10, 10
nonzero_elements = 10

# Initialize a sparse matrix with all zeros
matrix = np.zeros((rows, cols))

# Randomly generate indices for nonzero elements
row_indices = np.random.choice(rows, nonzero_elements, replace=True)
col_indices = np.random.choice(cols, nonzero_elements, replace=True)

# Generate random values for the nonzero elements
values = np.random.random(nonzero_elements)

# Populate the matrix
for r, c, v in zip(row_indices, col_indices, values):
    matrix[r, c] = v

orig_matrix = NDArray(matrix, device=device)
orig_matrix

/content/drive/MyDrive/10714/final_proj/gnn_with_spmm/python/needle


NDArray([[0.         0.         0.         0.         0.         0.
  0.07103606 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.7991586  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.87001216 0.0202184  0.        ]
 [0.         0.46147937 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.9786183  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.83261985 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.   

#### Then, we are going to transform it to a sparse matrix.

In [ ]:
sparse_matrix = orig_matrix.to_sparse()
sparse_matrix

SparseMatrix(nnz=8, shape=(10, 10),
  data=[0.07103606 0.7991586  0.87001216 0.0202184  0.46147937 0.9786183
 0.83261985 0.77815676],
  row_indices=[0 2 3 3 4 5 7 9],
  col_indices=[6 8 7 8 1 7 1 6])

As you can see, the sparse matrix contains three length 10 array, `data`, `row_indices` and `col_indices`. The `data` represents the values of the all the non-zero elements inside the matrix, while `row_indices` and `col_indices` represents the row and column index of each non-zero element's index.

We can also switch the sparse matrix back to dense matrix by calling `to_dense()` function.



In [ ]:
dense_matrix = sparse_matrix.to_dense()
dense_matrix

NDArray([[0.         0.         0.         0.         0.         0.
  0.07103606 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.7991586  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.87001216 0.0202184  0.        ]
 [0.         0.46147937 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.9786183  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.83261985 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.   

## 4. Sparse Matrix Multiplication
### COO Format Matrix Multiplication

Matrix multiplication in the **COO (Coordinate)** format involves multiplying two sparse matrices that are represented in the COO format. Since the COO format only stores nonzero elements along with their row and column indices, performing multiplication requires processing each nonzero element and its corresponding coordinates.

#### Step 1: Represent the Matrices in COO Format
Assume two matrices A and B, both stored in COO format. They are represented by the following components:

- **Matrix A**:
  - `A_data`: Nonzero values in matrix A.
  - `A_row`: Row indices of the nonzero values in A.
  - `A_col`: Column indices of the nonzero values in A.

- **Matrix B**:
  - `B_data`: Nonzero values in matrix B.
  - `B_row`: Row indices of the nonzero values in B.
  - `B_col`: Column indices of the nonzero values in B.

Let matrix A be of size m × n and matrix B be of size n × p. The resulting matrix C will be of size m × p.

#### Step 2: Initialize the Resultant Matrix
The resulting matrix C will also be sparse and initially contain only zeros. In COO format, the result will have:

- `C_data`: Nonzero values of the resulting matrix.
- `C_row`: Row indices of the nonzero values in C.
- `C_col`: Column indices of the nonzero values in C.

#### Step 3: Compute Nonzero Elements of the Result
To calculate C = A × B, follow these steps for each nonzero element of A:

1. **Find the corresponding element in matrix B**: For each nonzero element A_ij in A, find the corresponding column indices of B. The row index in B must match the column index in A to compute the dot product.

2. **Perform the multiplication**: For each pair of nonzero elements A_ij and B_jk, multiply them together and add to the corresponding entry in C:
   
   ```
   C_ik = C_ik + A_ij × B_jk
   ```

3. **Store the result**: If C_ik is nonzero after the above addition, store the result in the COO format:
   - Add the value of C_ik to `C_data`.
   - Add the row index i to `C_row`.
   - Add the column index k to `C_col`.

#### Step 4: Handle Sparse Properties
Since the result matrix C is also sparse, ensure that only nonzero values are stored. If the sum of C_ik is zero, it should not be stored in the COO format.
#### Define two sparse matrices.


In [ ]:
m, n, p = 500, 500, 500
device = cuda()
nnz = 100
# Initialize a sparse matrix with all zeros
matrix1 = np.zeros((m, n))
matrix2 = np.zeros((n, p))

# Randomly generate indices for nonzero elements
row_indices_1 = np.random.choice(m, nnz, replace=True)
col_indices_1 = np.random.choice(n, nnz, replace=True)
row_indices_2 = np.random.choice(n, nnz, replace=True)
col_indices_2 = np.random.choice(p, nnz, replace=True)

# Generate random values for the nonzero elements
values_1 = np.random.random(nnz)
values_2 = np.random.random(nnz)

# Populate the matrix
for r, c, v in zip(row_indices_1, col_indices_2, values_1):
    matrix1[r, c] = v
for r, c, v in zip(row_indices_2, col_indices_2, values_2):
    matrix2[r, c] = v

dense_matrix1 = NDArray(matrix1, device=device)
dense_matrix2 = NDArray(matrix2, device=device)
dense_matrix1, dense_matrix2

(NDArray([[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]], device=cuda()),
 NDArray([[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]], device=cuda()))

In [ ]:
# Convert dense matrices to sparse matrices.
sparse_matrix1 = dense_matrix1.to_sparse()
sparse_matrix2 = dense_matrix2.to_sparse()
sparse_matrix1, sparse_matrix2

(SparseMatrix(nnz=100, shape=(500, 500),
   data=[0.10029394 0.46357542 0.05537432 0.76532525 0.37416998 0.35561273
  0.6304479  0.29214752 0.4783703  0.40724117 0.3960597  0.58641016
  0.13248764 0.96157014 0.6178767  0.23170163 0.79920256 0.07952208
  0.96193635 0.77058077 0.1314828  0.29302028 0.14484777 0.49739137
  0.97749513 0.18327984 0.42468548 0.9473706  0.63947254 0.21550767
  0.09784448 0.22431703 0.8638556  0.72559434 0.11753186 0.05342718
  0.20747007 0.24536721 0.7168597  0.82211775 0.14694664 0.01323686
  0.27762872 0.9065555  0.8621915  0.58678436 0.9037197  0.9295293
  0.08960304 0.14814086 0.02566272 0.87650526 0.08342244 0.9608347
  0.21331197 0.5654213  0.42053947 0.06395527 0.8605512  0.23223414
  0.66991657 0.3472335  0.511319   0.55219245 0.08110139 0.7270443
  0.4856276  0.26211816 0.13690028 0.13206811 0.6720478  0.33314514
  0.01642963 0.2703279  0.58447605 0.9493188  0.94043195 0.87428796
  0.7308558  0.7740473  0.68328136 0.9413777  0.5573688  0.48805627
  0

#### Matrix Multiplication between dense matrices.

In [ ]:
import time
start = time.time()
correct_result = dense_matrix1 @ dense_matrix2
end = time.time()
print(f"Time taken for dense-dense matrix multipliction: {(end - start)*1000} ms")
correct_result

Time taken for dense-dense matrix multipliction: 0.3287792205810547 ms


NDArray([[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], device=cuda())

#### Matrix Multiplication between sparse matrices.

In [ ]:
start = time.time()
result = sparse_matrix1 @ sparse_matrix2
print(result)
# result = result.to_dense() # when flag = True return dense
end = time.time()
print(f"Time taken for sparse-sparse matrix multipliction: {(end - start)*1000} ms")
print(f"Result Correction: \n{np.allclose(correct_result.numpy(), result.numpy())}")

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Time taken for sparse-sparse matrix multipliction: 3.0074119567871094 ms
Result Correction: 
True


In [ ]:
start = time.time()
result = sparse_matrix1 @ dense_matrix2
result = result
end = time.time()
print(f"Time taken for sparse-dense matrix multipliction: {(end - start)*1000} ms")
print(f"Result Correction: \n{np.allclose(correct_result.numpy(), result.numpy())}")

Time taken for sparse-dense matrix multipliction: 0.49948692321777344 ms
Result Correction: 
True


In [ ]:
start = time.time()
result = dense_matrix1 @ sparse_matrix2
result = result
end = time.time()
print(f"Time taken for dense-sparse matrix multipliction: {(end - start)*1000} ms")
print(f"Result Correction: \n{np.allclose(correct_result.numpy(), result.numpy())}")

Time taken for dense-sparse matrix multipliction: 0.3407001495361328 ms
Result Correction: 
True


In [ ]:
!python -m backend_ndarray.ndarray

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'backend_ndarray.ndarray' found in sys.modules after import of package 'backend_ndarray', but prior to execution of 'backend_ndarray.ndarray'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
Time for dense @ dense: 0.225067138671875 ms
Time for dense @ sparse: 0.9086132049560547 ms
Time for sparse @ dense: 0.2598762512207031 ms
Time for sparse @ sparse: 0.09012222290039062 ms
Total time: 1.2586116790771484 ms
dense @ sparse: True
sparse @ dense: True
sparse @ sparse: True


## Graph Neural Network(GNN)
A Graph Neural Network (GNN) is a type of neural network specifically designed to process and analyze data structured as graphs. In a graph, data is represented as nodes (vertices) connected by edges (links), which may have associated features or attributes.
### Common GNN Architectures:
1. Graph Convolutional Networks (GCN): Generalizes the concept of convolution to graphs.
2. Graph Attention Networks (GAT): Uses attention mechanisms to weigh neighbor contributions.
3. GraphSAGE: Focuses on efficient sampling and aggregating information from neighbors.

In this project, we are going to implement Graph Convolutional Network(GCN) and use sparse matrix multiplication mechnisms we implemented to accelerate its forward and backward passes.